In [5]:
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd
df = pd.read_csv('QandA_list.csv', encoding='utf-8')
answers = df["Question answers"].tolist()
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)

openai.api_key = token=os.environ['CHAT_TOKEN']

In [3]:
"""
Basic info:
- model variable defines what model to use. Be careful as different models cost differetly
- ChatCompletetion.create() has 3 main roles:
    - system: meta-prompt, basically what the software engineers want the model to do
    - user: end user input, free form. keep in mind prompt jailbreaking when designing prompts
    - assistant: used to plug in arbitrary model output. basically a gas lighting tool, also
    useful for creating conversational memory

Meta prompt:
- Defines basic behavior of model, if calling of functions or returning json output is 
desired, you have to *verbally* convince the models to use the functions. There is no
guarantee, and god knows I have no idea how openAI implements it in the back end

Function:
- Not a real function, but a mental? verbal? (who knows) algorithm for the model to undergo
when doing inference. Model will (or will not) call function depending on wording of meta prompt.
Outputs are always defined, though not necessarily will be returned (model will sometimes
empty jsons for example). To make it work, use prompt engineering.
- For reference, removing "use only the 'classify' function" from the meta prompt actually
*improved* consistency of the model using the function correctly.
- Note: There can be many functions

Todo:
- Return all model outputs and data (load time, token count, etc) to a json file for 
documentation and more r&d.

--- Everything here is under heavy experimentation, no one knows what's going on and nothing 
makes any sense ---
"""
def doorman(prompt):
    #Load meta prompt
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    #Define API Call
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = [
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": prompt},
        ],
        #Define 'function' for model to call during execution
        functions = [
            {
                "name": "classify",
                #Verbal description to help the model use the function correctly,
                #for the time being, telling the model to return 0 if no matches are found
                #seems to do absolutely nothing
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                #This actually does absolutely nothing, model frequently returns empty json
                #anyway, needs further testing
                "required": ["group"],
            },
        ],
        #Define model parameters
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            return int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
        else:
            return 0
    except:
        return 0

In [4]:
doorman("I'm cold in my office")

4

In [6]:
with open('llmlads_prompt.txt','r') as file:
    meta_prompt = file.read()
print(meta_prompt)

Given the following group of statements, classify a user statement into most similar group, if there are no matches, output 0 for question group, use the "classify" function provided:
List of groups: 
1: How to request a book purchase
How do I buy books for my lab? 
I need to purchase a textbook from amazon. How to start?
2: Who should I contact for graduate students related questions?
Questions about graduate students at CS department 
3: How do I get reimbursed for a work expense?
How do I request to have an invoice paid by TReNDS? 
How do I request a work expense to be paid by purchase card (P-Card)? 
Who should I contact for a publication fee reimbursement? 
How does my student get reimbursed for a conference fee?
Please reimburse my fMRI course
4: It is too cold in my office. Who do I contact?
It is too hot in my office. Who do I contact?
There are bugs in the kitchen. Who should I report this to?
How should I report a clogged sink?
Who do I complain about dirty dishes?
I need to 

In [38]:
import numpy as np
def doorman_with_memory(prompt, log):
    #Load meta prompt
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    #Define API Call
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = messages,
        #Define 'function' for model to call during execution
        functions = [
            {
                "name": "classify",
                #Verbal description to help the model use the function correctly,
                #for the time being, telling the model to return 0 if no matches are found
                #seems to do absolutely nothing
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                #This actually does absolutely nothing, model frequently returns empty json
                #anyway, needs further testing
                "required": ["group"],
            },
        ],
        #Define model parameters
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            return int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
        else:
            return 0
    except:
        return 0

In [ ]:
import numpy as np
import json
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd

def write_json_to_file(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)

def load_json_from_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['logs']

def append_log(logs, t_id, role, content):
    for log in logs:
        if (log['t_id'] == t_id):
            log['thread_log'].append({"role": role, "content": content})
            return logs
    new_log = {
        "t_id": t_id,
        "thread_log": [
            {"role": role, "content": content}
        ],
    }
    logs.append(new_log)
    return logs
def load_log(logs, t_id):
    for log in logs:
        if (log['t_id'] == t_id):
            return log['thread_log']
    return []

def doorman_v0(logs, t_id, prompt, answers):
    # Function params, mess around with these to experiment with model behvior
    null_response = "I'm sorry, could you provide more clarifying information?"
    max_tokens = 100
    temperature = 0.8
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    # Load thread log if exists
    system_prompt = np.array([{"role": "system", "content": meta_prompt}])
    user_prompt = np.array([{"role": "user", "content": prompt}])
    thread_log = np.array(load_log(logs, t_id))
    messages = np.concatenate((system_prompt, thread_log, user_prompt)).tolist()
    #print(messages)
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-16k",
        messages = messages,
        functions = [
            {
                "name": "classify",
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                "required": ["group"],
            },
        ],
        function_call="auto",
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            index = int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
            answer = answers[index - 1]
            logs = append_log(logs, t_id, "user", prompt)
            logs = append_log(logs, t_id, "assistant", answer)
            return index
        else:
            logs = append_log(logs, t_id, "user", prompt)
            logs = append_log(logs, t_id, "assistant", null_response)
            return 0
    except:
        logs = append_log(logs, t_id, "user", prompt)
        logs = append_log(logs, t_id, "assistant", null_response)
        print("<dev flag> Function wasn't even called lol")
        return 0
    
################################################
#### surround this code with __name__ block ####
################################################
if __name__ == "__main__":
    df = pd.read_csv('QandA_list.csv', encoding='utf-8')
    answers = df["Question answers"].tolist()
    env_path = Path('.')/'.env'
    load_dotenv(dotenv_path=env_path)
    openai.api_key = token=os.environ['CHAT_TOKEN']
    try:
        logs = load_json_from_file("doorman_log.json")
        while(True):
            t_id = int(input("Thread ID: "))
            prompt = input("User prompt: ")
            index = doorman_v0(logs, t_id, prompt, answers)
            if index == 0:
                print("Not found")
            else:
                print("Found")
                print(answers[index-1])
            write_json_to_file({"logs" :logs}, "doorman_log.json")
    except:
        if os.path.isfile("doorman_log.json"):
            print("<dev flag> something went wrong, but doorman_log.json exists")
        else:
            print("<dev flag> doorman log not found, making empty file called doorman_log.json")
            logs = {"logs" : []}
            write_json_to_file(logs, "doorman_log.json")
################################################
################################################
################################################

Thread ID: 123456
User prompt: jalsenewlf
Not found
Thread ID: 123456
User prompt: I'm kinda cold
Found
Please submit a TReNDS Facilities Report Form here. If the situation is a facility emergency, please email ops@trendscenter.org or find a member of the Operations Team.
Thread ID: 123456
User prompt: do we have a email lisintg
Found
A TReNDS Center specific directory can be found here on the intranet. The GSU campus directory can be found here.  You can also find people by task on the trends intranet.


In [104]:
log1 = {
    "t_id" : 123456,
    "thread_log": [
        {"role": "user", "content": "hot"},
        {"role": "assistant", "content": "I'm sorry, could you provide more clarifying information?"},
    ],
}
log2 = {
    "t_id" : 120395,
    "thread_log": [
        {"role": "user", "content": "cold"},
        {"role": "assistant", "content": "I'm sorry, could you provide more clarifying information?"},
    ],
}
logs = [log1, log2]
def append_log(logs, t_id, role, content):
    for log in logs:
        if (log['t_id'] == t_id):
            log['thread_log'].append({"role": role, "content": content})
            return logs
    new_log = {
        "t_id": t_id,
        "thread_log": [
            {"role": role, "conent": content}
        ],
    }
    logs.append(new_log)
    return logs
def load_log(logs, t_id):
    for log in logs:
        if (log['t_id'] == t_id):
            return log['thread_log']
    return []

In [101]:
index = doorman_v0(logs, 123456, "international flight", answers)
if index == 0:
    print("Statement not found")
else:
    print(answers[index-1])

<strong>Process for the</strong> <strong>Traveler</strong><br>
Travelers should complete the Travel Authorization that indicates their GSU affiliation (Employee, Student, or Non - Employee). The traveler should refer to <a href="https://nam11.safelinks.protection.outlook.com/?url=https%3A%2F%2Fservices.gsu.edu%2Ftravel%2F&amp;data=05%7C01%7Csplis%40gsu.edu%7C33c5802965554500328608da292c121f%7C515ad73d8d5e4169895c9789dc742a70%7C0%7C0%7C637867566510080641%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&amp;sdata=dgBquAMceRvK67coHvmfUGdaxyAGAKcR6sBUWmuqTPE%3D&amp;reserved=0" rel="noopener noreferrer" target="_blank" title="Original URL: https://services.gsu.edu/travel/. Click or tap if you trust this link.">https://services.gsu.edu/travel/</a> and select Travel Inc. Website for guidance with the GSU travel process. Initiation of travel requests should start one to two months prior to the date of travel. Later requests may not be e

In [102]:
for i in logs:
    print(i)

{'t_id': 123456, 'thread_log': [{'role': 'user', 'content': 'hot'}, {'role': 'assistant', 'content': "I'm sorry, could you provide more clarifying information?"}, {'role': 'user', 'content': 'international flight'}, {'role': 'assistant', 'content': '<strong>Process for the</strong> <strong>Traveler</strong><br>\nTravelers should complete the Travel Authorization that indicates their GSU affiliation (Employee, Student, or Non - Employee). The traveler should refer to <a href="https://nam11.safelinks.protection.outlook.com/?url=https%3A%2F%2Fservices.gsu.edu%2Ftravel%2F&amp;data=05%7C01%7Csplis%40gsu.edu%7C33c5802965554500328608da292c121f%7C515ad73d8d5e4169895c9789dc742a70%7C0%7C0%7C637867566510080641%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&amp;sdata=dgBquAMceRvK67coHvmfUGdaxyAGAKcR6sBUWmuqTPE%3D&amp;reserved=0" rel="noopener noreferrer" target="_blank" title="Original URL: https://services.gsu.edu/travel/. Click or tap i

In [ ]:
app = Flask(__name__)
slack_event_adapter = SlackEventAdapter(
    os.environ['SIGNING_SECRET'], '/slack/events',app)

client = slack.WebClient(token=os.environ['SLACK_TOKEN'])
#response = client.api_call("auth.text")
#BOT_ID = response['user_id']

@slack_event_adapter.on('message')
def message(payload):
    event = payload.get('event',{})
    channel_id = event.get('channel')
    user_id = event.get('user')
    text = event.get('text')
    inp = "User input: " + "\"" + text + "\""
    if user_id != 'U05EY26K3FZ':
        index = doorman(inp)
        if index == 0:
            text = "Yaabaadabaadoo"
            client.chat_postMessage(channel=channel_id, text=text)
        else:
            text = answers[index - 1]
            client.chat_postMessage(channel=channel_id, text=text)
            
app.run(debug=True)

In [ ]:
# Main tesing loop, repeatedly prompts user until user inputs "end_conversation"
inp = ""
while (True):
    inp = input("$ ")
    if (inp == "end_conversation"):
        break
    #This formats the prompt to as "User input 'prompt' ", unclear if it does anything
    inp = "User input: " + "\"" + inp + "\""
    index = doorman(inp)
    if index == 0:
        print("I'm sorry but could you provide me with a bit more clarification?")
    else:
        print(answers[index - 1])

In [2]:
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd
df = pd.read_csv('QandA_list.csv', encoding='utf-8')
answers = df["Question answers"].tolist()

env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)

openai.api_key = token=os.environ['CHAT_TOKEN']

def doorman(prompt):
    #Load meta prompt
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    #Define API Call
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = [
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": prompt},
        ],
        #Define 'function' for model to call during execution
        functions = [
            {
                "name": "classify",
                #Verbal description to help the model use the function correctly,
                #for the time being, telling the model to return 0 if no matches are found
                #seems to do absolutely nothing
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                #This actually does absolutely nothing, model frequently returns empty json
                #anyway, needs further testing
                "required": ["group"],
            },
        ],
        #Define model parameters
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            return int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
        else:
            return 0
    except:
        return 0

app = Flask(__name__)
slack_event_adapter = SlackEventAdapter(
    os.environ['SIGNING_SECRET'], '/slack/events',app)

client = slack.WebClient(token=os.environ['SLACK_TOKEN'])
#response = client.api_call("auth.text")
#BOT_ID = response['user_id']

@slack_event_adapter.on('message')
def message(payload):
    event = payload.get('event',{})
    channel_id = event.get('channel')
    user_id = event.get('user')
    text = event.get('text')
    inp = "User input: " + "\"" + text + "\""
    if user_id != 'U05EY26K3FZ':
        index = doorman(inp)
        if index == 0:
            text = "I'm sorry but could you provide me with a bit more clarification?"
            client.chat_postMessage(channel=channel_id, text=text)
        else:
            text = answers[index - 1]
            client.chat_postMessage(channel=channel_id, text=text)
            
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/usr/lib/python3/d

SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
